# **Mask operations**

Here we explore different operations with image masks and tricks that can be helpful in different contexts.

---

## **Preparations**

The usual preparations... Before we begin, let's load some drawing functions for rendering images effortlessly in this Jupyter notebook.

In [ ]:
import sys
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Functionality related to this course
sys.path.append("..")
import tools

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

Read the image data. We use the below image of red and white blood cells. We previously have created a segmentation mask for the red blood cells in the image.

In [ ]:
# The default dataset (feel free to change)
mask = cv.imread("../data/images/hematology-baso1-mask.png", cv.IMREAD_GRAYSCALE)
img = cv.imread("../data/images/hematology-baso1.jpg", cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
tools.show_image_chain([img, mask], titles=["Image", "Mask (red blood cells)"], suppress_info=True);

To demonstrate the effect of some operations, we will use a noisy version of that mask.

In [ ]:
# Noisy version of the mask
np.random.seed(1)
mask_noisy = mask.copy()
mask_noisy[np.random.rand(*mask_noisy.shape) < 0.1] = 0
tools.show_image(mask_noisy, suppress_info=True);

## **Hole filling**

Hole filling refers to the process of filling in holes in the mask. For small holes, we can use morphological operations. For larger holes, we can use the outer contours of the components and fill them in.

In [ ]:
# Method 1: Morphological operations
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
mask_filled1 = cv.morphologyEx(mask_noisy, cv.MORPH_CLOSE, kernel, iterations=2)


# Method 2: Contour filling
mask_filled2 = mask_noisy.copy()
contour, hierarchy = cv.findContours(mask_noisy, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
for cnt in contour:
    cv.drawContours(mask_filled2,[cnt],0,255,-1)


# Visalize the results
tools.show_image_chain([mask_noisy, mask_filled1, mask_filled2], 
                       titles=["Noisy mask (input)", "Filled mask (morphology)", "Filled mask (contour)"],
                       suppress_info=True);

## **Flood fill**

Imagine that we want to fill the mask from a certain pixel with a certain gray level (e.g., 255), but only if the pixel is connected to the top-left corner. This can be achieved using the flood fill algorithm.

In the below example, we start the flood fill from the pixel indicated by the red point (seed point) at pixel (30, 30).

In [ ]:
# Specify the seed point
seed_point = (30, 30)

# Flood fill
mask_filled = mask_noisy.copy()
cv.floodFill(mask_filled, None, seed_point, 255)

# Visualize the seed point as a red dot
mask_filled_with_seed = cv.cvtColor(mask_filled, cv.COLOR_GRAY2RGB)
cv.circle(mask_filled_with_seed, seed_point, 7, [255,0,0], -1)
tools.show_image_chain([mask_noisy, mask_filled_with_seed, None], 
                       titles=["Input", "Filled mask (flood fill)", None],
                       suppress_info=True);


One can use an additional mask to restrict the flood fill. Note that the restriction mask should be two pixels larger in each dimension than the image, a requirement of `cv.floodFill()`.

In [ ]:
# Apply flood fill with a restriction mask
mask_filled = mask_noisy.copy()
mask_restriction = np.zeros((mask_filled.shape[0]+2, mask_filled.shape[1]+2), dtype=np.uint8)
mask_restriction = cv.circle(mask_restriction, (150, 150), 125, 255, 3)
mask_restriction = cv.rectangle(mask_restriction, (250, 100), (450, 300), 255, 3)
mask_restriction = cv.ellipse(mask_restriction, (500, 330), (100, 80), 30, 0, 360, 255, 3)
cv.floodFill(mask_filled, mask_restriction, (0, 0), 255)

# Visualize the seed point as a red dot
mask_filled_with_seed = cv.cvtColor(mask_filled, cv.COLOR_GRAY2RGB)
cv.circle(mask_filled_with_seed, seed_point, 7, [255,0,0], -1)
tools.show_image_chain([mask_noisy, mask_restriction, mask_filled_with_seed], 
                       titles=["Input", "Mask", "Filled mask (flood fill)"],
                       suppress_info=True);

Note that in the previous example, the flood fill is obstructed not only by the mask, but also by the image content. This is why the flood fill does not fill the entire image. Only the pixels that are connected (= neighboring pixels with the same pixel value) to the seed point are filled.

## **Connected components**

Assume you have a binary segmentation mask with multiple objects in it. 
The goal is to find the number of objects and their bounding boxes.

We can use OpenCV's `cv.connectedComponents()` to label the connected components. The function returns an image with the same size as the input image, where each pixel has a label corresponding to the connected component it belongs to. Background pixels are labeled with zero.

In [ ]:
# Method 1: OpenCV
# Connectivity: 4 or 8. This parameter specifies how pixels are connected.
# In the 4-connected case, two pixels are connected if they share an edge.
# In the 8-connected case, two pixels are connected if they share an edge or a corner.
labels = cv.connectedComponents(mask, connectivity=8)[1]

And that's about it. The rest of this section is about how the various components can be visualized.

In [ ]:
# Container for visualization
results = {}
results["Input"] = mask

# Visualization 1: 
# ################
# Display the connected components in different colors.
def colorize_labels(labels):
    """
    Represent the data in HSV such that the connected components are colored
    differently. Recall: HSV refers to hue, saturation, and value. Hue 
    represents an angle in the color wheel, saturation the intensity of the 
    color, and value the brightness. To represent black, we set value to zero.
    """
    h = labels/labels.max()*128
    s = np.ones_like(labels)*255
    v = (labels > 0) * 255
    labels_color = np.stack([h, s, v], axis=-1)
    labels_color = cv.cvtColor(labels_color.astype(np.uint8),
                               cv.COLOR_HSV2RGB)
    return labels_color

labels_color = colorize_labels(labels)
results["Connected components"] = labels_color

# Visualization 2: 
# ################
# Restrict the number of colors and shuffle them. 
# We can use a look-up table (LUT) for this purpose.
def colorize_labels_random(labels):
    h_vals = [1, 34, 55, 99]  
    assert 0 not in h_vals  # Zero is reserved for the background
    # For reproducability
    np.random.seed(1)
    # Sample n times with replacement from h_vals 
    lut = np.random.choice(h_vals, labels.max())
    # Insert a zero at the beginning to represent the background
    lut = np.insert(lut, 0, 0)
    # Lookup values
    labels_new = lut[labels]
    labels_color = colorize_labels(labels_new)
    return labels_color

labels_color = colorize_labels_random(labels)
results["Randomized colors"] = labels_color

tools.show_image_grid(results, figsize=(10, 10), suppress_info=True);

The above code is powered by `cv.connectedComponents()`. Note that we can also use  `scipy.ndimage.label()` for this purpose. The latter is more flexible and can be used for n-dimensional data.

In [ ]:
import scipy.ndimage as si
labels, n_labels = si.label(mask)

# It's output is equivalent to cv.connectedComponents().
ret = colorize_labels(labels)

# Compare the results
tools.show_image_chain([results["Connected components"], ret],
                       titles=["OpenCV: cv.connectedComponents()", 
                               "Scipy: scipy.ndimage.label()"], 
                       suppress_info=True);

We may want to calculate different properties for each of the extracted components. Before we do that, let's first introduce some other concepts such as the contour or convex hull of a shape that will be useful for this purpose.

## **Contours**

Contours are the boundaries of connected components. We can extract contours from a binary image directly using `cv.findContours()`.

The function returns two values: a list of contours and hierarchy information. The hierarchy is a list of four indices that describe the relationship between the contours: [next, previous, first child, parent]. Each of these four values is an index that identifies the corresponding contour. If there is no such element, the value is -1. 

In the following, we will not use the hierarchy information, so we will ignore the second return value. Hierarchy information is useful when contours are nested, i.e. when there are contours within other contours. 

There are different modes to retrieve the contours (argument `mode`). The most common ones are:
- `cv.RETR_EXTERNAL`: Only the external contours are returned.
- `cv.RETR_LIST`: All contours are returned in a flat list 
- `cv.RETR_CCOMP`: All contours are returned in a two-level hierarchy (external and holes)
- `cv.RETR_TREE`: All contours are returned in a tree hierarchy

Finally, the parameter `method` specifies how the contours are approximated (or simplified). The main goal here is to reduce the number of points while maintaining the shape of the contour.


In [ ]:
contours, _ = cv.findContours(mask, 
                              mode=cv.RETR_EXTERNAL, 
                              method=cv.CHAIN_APPROX_SIMPLE)

With `cv.drawContours()` we can draw (and fill) contours conveniently. In the following, we demonstrate different ways to visualize contours.

In [ ]:
results = {}
results["Input"] = mask

# Visualization 1: Simple contours
# ################################
img_outlines = np.zeros_like(img)
color = [255, 255, 0]
cv.drawContours(img_outlines, contours, 
                contourIdx=-1, 
                color=color, 
                thickness=2)
results["Plain contours"] = img_outlines

# Visualization 2: Single contour (filled)
# ########################################
img_outlines = np.zeros_like(img)
color = [50, 100, 255]
contour_id = 42
cv.drawContours(img_outlines, contours, 
                contourIdx=contour_id, 
                color=color, 
                thickness=-1)
results["Single contour (filled)"] = img_outlines

# Visualization 3: Use colors
# ###########################
import matplotlib as mpl
# Use colormaps from matplotlib
# https://matplotlib.org/stable/users/explain/colors/colormaps.html
cmap = mpl.colormaps["inferno"]
colors = [cmap(i) for i in np.linspace(0, 1, len(contours))]
# Convert colors from [0, 1] to [0, 255]
colors = [[int(c*255) for c in color] for color in colors]
img_outlines = img.copy()
for i, contour in enumerate(contours):
    cv.drawContours(img_outlines, 
                    contours, 
                    contourIdx=i, 
                    color=colors[i], 
                    thickness=3)
results["Contours in color"] = img_outlines

# Visualization 4: Random colors
# ##############################
img_outlines = img.copy()
contours_shuffled = list(contours)
#np.random.shuffle(contours_shuffled)
colors = [[50, 100, 255],
          [255, 100, 50],
          [50, 255, 100]]
for i, contour in enumerate(contours_shuffled):
    cv.drawContours(img_outlines, 
                    contours_shuffled, 
                    contourIdx=i, 
                    color=colors[i % len(colors)], 
                    thickness=3)
results["Randomized colors"] = img_outlines

# Visualization 5: Colorize based on size
# #######################################
sizes = np.array([cv.contourArea(contour) for contour in contours])
sizes = sizes/sizes.max()  # Normalized sizes
img_outlines = img.copy()
# Use the following line if you want to sort the contours by size
# contours_sorted = sorted(contours, key=cv.contourArea)

# Use the colormap "hsv". It contains a transition red-yellow-green.
# cmap is a function that maps a scalar between 0 and 1 to a color.
# To sample only colors in the red-yellow-green range, we can use only
# the first 30% of the colormap. 
cmap = mpl.colormaps["hsv"]
colors = [cmap(s*0.3) for s in sizes]
colors = [[int(c*255) for c in color] for color in colors]

for i, contour in enumerate(contours):
    cv.drawContours(img_outlines, 
                    contours, 
                    contourIdx=i, 
                    color=colors[i % len(colors)], 
                    thickness=-1)  # Filled
results["Colors sorted by size"] = img_outlines

# Visualization 6: Bounding boxes and centroids
# #############################################
def draw_bounding_boxes(img, contours):
    assert img.ndim == 3
    img = img.copy()
    cmap = mpl.colormaps["hsv"]
    colors = [cmap(i) for i in np.linspace(0, 1, len(contours))]
    colors = [[int(c*255) for c in color] for color in colors]
    for i, contour in enumerate(contours):
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(img, (x, y), (x+w, y+h), 
                    color=colors[i], thickness=2)
        
        # For the center of mass, we need to compute the moments of the contour
        # https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html
        M = cv.moments(contour)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv.circle(img, (cx, cy), 5, color=colors[i], thickness=-1)
    return img

img_in = cv.cvtColor(mask, cv.COLOR_GRAY2RGB)
ret = draw_bounding_boxes(img_in, contours)
results["Bounding boxes"] = ret

ret = draw_bounding_boxes(img, contours)
results["Bounding boxes (as overlay)"] = ret

tools.show_image_grid(results, figsize=(10, 9), suppress_info=True);

## **Convex hull**

A convex object is a set of points where the line segment connecting *any two points*
in the set lies always entirely within the set. See below for examples of convex and
non-convex objects.

![Convex and non-convex shapes](../data/doc/convexity.svg)

**Figure**: Illustration of non-convex (left) and convex shapes (middle and right). Image source: [Link](https://d2l.ai/index.html)

The **convex hull** of a geometric object (or a set of points) is the smallest
convex shape that contains the object. The convex hull of a set of points
can be computed using the OpenCV function `cv.convexHull()`.


In [ ]:
# Compute the convex hull for a binary mask
mask_text = cv.imread("../data/images/word-ice-cream.png", cv.IMREAD_GRAYSCALE)
mask_text = 255 - mask_text  # Invert the mask
contours_text, _ = cv.findContours(mask_text, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

drawing = cv.cvtColor(mask_text, cv.COLOR_GRAY2RGB)
color = (255, 0, 0)
for i in range(len(contours_text)):
    hull = cv.convexHull(contours_text[i])
    cv.drawContours(drawing, [hull], 0, color=color, thickness=1)

tools.show_image_chain([mask_text, drawing], ncols=1,
                       titles=["Input", "Convex hull"], 
                       suppress_info=True, 
                       figsize=(6, 5));


For our initial mask with the red blood cells, the convex hulls look as follows:

In [ ]:
# Compute the convex hull for a binary mask
drawing = img.copy()
color = (255, 255, 0)
for i in range(len(contours)):
    hull = cv.convexHull(contours[i])
    cv.drawContours(drawing, [hull], 0, color=color, thickness=2)

tools.show_image_chain([mask, drawing], 
                       titles=["Input", "Convex hull"], 
                       suppress_info=True, 
                       figsize=(8, 5));

## **Distance transform**

The distance transform computes the distance of each pixel to the nearest zero pixel. The distance transform can be useful for shape analysis or preprocessing.

Below we demonstrate how the distance transformations are calculated for two different mask images.

In [ ]:
results = {}
results["Input 1"] = mask
results["Input 2"] = mask_text

# Compute the distance transform
dist_transform = cv.distanceTransform(mask, cv.DIST_L2, cv.DIST_MASK_PRECISE)
dist_transform /= dist_transform.max()
results["Distance transform 1"] = dist_transform

dist_transform = cv.distanceTransform(mask_text, cv.DIST_L2, cv.DIST_MASK_PRECISE)
dist_transform /= dist_transform.max()
results["Distance transform 2"] = dist_transform

tools.show_image_grid(results, ncols=2, figsize=(10, 5), suppress_info=True, shape=None);


The distance transform results in a grayscale image that represents the distance to the nearest zero pixel. The brighter the pixel, the further away it is from the nearest zero pixel.

## **Thinning**
Thinning (or skeletonization) is the process of reducing a binary image to a skeleton representation. The skeleton is a thin representation of the object that is useful for shape analysis. It can be computed using the Zhang-Suen algorithm ([link](https://doi.org/10.1145/357994.358023)), which is implemented in the opencv-contrib-python package.

In [ ]:
# Compute the skeleton (requires the opencv-contrib-python package)
if (not hasattr(cv, "ximgproc")) or (not hasattr(cv.ximgproc, "thinning")):
    raise RuntimeError("This version of OpenCV does not support thinning.")


thinned = cv.ximgproc.thinning(mask_text)
ret = cv.cvtColor(mask_text, cv.COLOR_GRAY2RGB)
ret[thinned == 255] = [0, 0, 255]

tools.show_image_chain([mask_text, ret], ncols=1, 
                       titles=["Input (inverted mask)", 
                               "Skeletonization of the mask"], 
                       suppress_info=True, figsize=(6, 5));



We can use thinning not only to reduce a binary image to a skeleton, but also
find paths that are equidistant to the boundaries of the objects in the image. 
You can see an example below. Note that we calculate the thinning for the background mask: `mask_bg = (255-mask)`. 

The lines in the below example indicate pixels that are at the same distance from (at least) two contours.

In [ ]:
# Compute thinning on the background mask
thinned = cv.ximgproc.thinning(255-mask)
ret = img.copy()
ret[thinned == 255] = [0, 0, 0]

# Visualize the results
tools.show_image_chain([mask, ret], ncols=2, 
                       titles=["Input (inverted mask)", 
                               "Skeletonization of the inverted mask"], 
                       suppress_info=True, figsize=(7, 12));


## **Measuring components**

We sometimes need to compute geometric properties per component or contour. For instance, we may want to compute the center of mass, area, perimeter, circularity, etc. for each of the red blood cells in the mask in our sample dataset. The following code demonstrates how to compute these properties.

Circularity $c$ measures how close the shape of an object is to a circle. It is defined as 
$$c = \frac{4\cdot \pi\cdot A}{p^2}$$
Note that that the circle is the shape with the smallest perimeter for a given area, which is why the circularity $c\leq 1$, where $c=1$ for a perfect circle. We can use this metric to distinguish rather round objects from elongated or irregular shapes. 

Of course, we can calculate many other metrics that describe the shape of an object (so-called shape descriptors). Such parameters are usually very domain-specific.

Note that component 0 in the below result table refers to the background.

In [ ]:
# Compute the area of each connected component
areas = np.bincount(labels.flatten())
n_labels = labels.max()

# Compute the perimeter of each connected component
perimeters = np.zeros(n_labels+1)
areas_1 = np.zeros(n_labels+1)
areas_2 = np.zeros(n_labels+1)
circularities = np.zeros(n_labels+1)
centers = np.zeros((n_labels+1, 2))

for i in range(1, n_labels+1):
    mask_i = (labels == i).astype(np.uint8)
    contours, _ = cv.findContours(mask_i, 
                                  cv.RETR_EXTERNAL, 
                                  cv.CHAIN_APPROX_SIMPLE)
    assert len(contours) == 1  # We expect only one contour
    # Perimeter
    perimeters[i] = cv.arcLength(contours[0], closed=True)
    
    # Area of the contour (measured using the Green's theorem)
    areas_1[i] = cv.contourArea(contours[0])
    # Area of the contour (measured as the number of pixels)
    areas_2[i] = np.sum(mask_i)
    # Note: The two area measures are not always the same!
    
    # Circularity 
    circularities[i] = 4*np.pi*areas[i]/(perimeters[i]**2)
    
    # Center of mass
    centers[i, :] = np.mean(np.argwhere(mask_i), axis=0)

# Collect the info in a pandas DataFrame
df = pd.DataFrame({"Area": areas, 
                   "Perimeter": perimeters, 
                   "Circularity": circularities,
                   "CenterX": centers[:, 1], 
                   "CenterY": centers[:, 0]})
df.index.name = "Component"
# Sort values by area
df = df.sort_values(by="Area", ascending=False)

display(df.head(20))


## **Evaluation metrics**

It is sometimes required to compare two masks. For instance, in the context of image segmentation,
one might want to compare the ground truth mask with the predicted mask. Or one might want to compare
two different structures in an image for their similarity. 

In the following, we are going to learn how to compare two masks using different metrics.

Let's imagine that we have two masks $X$ and $Y$ that represent two shapes. Furthermore, let's assume that both masks consist of only one connected component, as illustrated below.

We are going to work with the following popular metrics to compare two binary masks $X$ and $Y$:
- [Dice coefficient](https://en.wikipedia.org/wiki/Dice-Sørensen_coefficient): 
     - in words: the size of the intersection divided by the average size of the two sets
     - in formula: $D(X, Y) = \frac{2|X \cap Y|}{|X| + |Y|}$
- [Intersection over Union (IoU)](https://en.wikipedia.org/wiki/Jaccard_index): 
     - in words: the size of the intersection divided by the size of the union
     - in formula: $IoU(X, Y) = \frac{|X \cap Y|}{|X \cup Y|}$ (also known as the Jaccard index)
- [Hausdorff distance](https://en.wikipedia.org/wiki/Hausdorff_distance):
     - in words: the largest minimal distance from a point in $X$ to a point in $Y$ or vice versa
     - in formula: $H(X, Y) = \max(\max_{x \in X} \min_{y \in Y} d(x, y), \max_{y \in Y} \min_{x \in X} d(x, y))$

In [ ]:
# Draw a circle
mask1 = np.zeros((300, 300), dtype=np.uint8)
cv.circle(mask1, (150, 150), 120, 255, -1)

# Draw a square.
mask2 = np.zeros((300, 300), dtype=np.uint8)
cv.rectangle(mask2, (50, 50), (250, 250), 255, -1)

# Comparison
mask_combine = np.zeros((300, 300, 3), dtype=np.uint8)
mask_combine[..., 0] = mask1
mask_combine[..., 1] = mask2

tools.show_image_chain([mask1, mask2, mask_combine], 
                       titles=["Mask 1", "Mask 2", "Comparison"], 
                       suppress_info=True, figsize=(6, 3));

In [ ]:
# Compute the dice coefficient
def dice(mask1, mask2):
    """Compute the Dice coefficient. The input masks should be binary."""
    assert mask1.shape == mask2.shape
    assert mask1.dtype == bool and mask2.dtype == bool
    intersection = mask1 & mask2  # Bitwise AND, equivalent to np.logical_and()
    return 2*np.sum(intersection)/(np.sum(mask1) + np.sum(mask2))


def iou(mask1, mask2):
    assert mask1.shape == mask2.shape
    assert mask1.dtype == bool and mask2.dtype == bool
    intersection = mask1 & mask2  # Bitwise AND, equivalent to np.logical_and()
    union = mask1 | mask2         # Bitwise OR, equivalent to np.logical_or()
    return np.sum(intersection)/np.sum(union)


def hausdorff(mask1, mask2):
    assert mask1.shape == mask2.shape
    from scipy.spatial import distance
    ij1 = np.argwhere(mask1)  # Indices of the non-zero elements
    ij2 = np.argwhere(mask2)  # Ditto
    # Note: In general, the directed maximal distance from mask1 to mask2 is 
    #       not the same as the directed maximal distance from mask2 to mask1.
    #       We take the maximum of the two.
    ret12, _, _ = distance.directed_hausdorff(ij1, ij2)
    ret21, _, _ = distance.directed_hausdorff(ij2, ij1)
    return max(ret12, ret21)


# Binarize the masks
mask1 = mask1 > 0
mask2 = mask2 > 0

scores = {}
scores["Dice"] = dice(mask1, mask2)
scores["IoU"] = iou(mask1, mask2)
scores["Hausdorff"] = hausdorff(mask1, mask2)

print("Scores for the two masks:")
print("="*25)
print(pd.Series(scores).to_string())


The above results measure a good, but not perfect overlap.

Let's now apply the comparison of masks to the red blood cells. To illustrate the process, we will first compute an alternative version of the segmentation mask we have used so far of the red blood cells.

In [ ]:
# Threshold the image (Otsu's method)
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
_, mask_test = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
# Get rid of segmentation noise using morphological closing
#mask_test = cv.morphologyEx(mask_test, cv.MORPH_CLOSE, kernel, iterations=2)

# Let's visualize the original image and the two masks
tools.show_image_chain([img, mask, mask_test], 
                       titles=["Input", "Ground truth", "Otsu's method"], 
                       suppress_info=True, figsize=(9, 3));


In [ ]:
# Compute the overall scores
mask_bin = mask > 0
mask_test_bin = mask_test > 0

scores_overall = {}
scores_overall["Dice"] = dice(mask_bin, mask_test_bin)
scores_overall["IoU"] = iou(mask_bin, mask_test_bin)
# --> This score is computationally expensive!
scores_overall["Hausdorff"] = hausdorff(mask_bin, mask_test_bin)
print("Overall scores:")
print("="*25)
print(pd.Series(scores_overall).to_string())

Let's now try to identify those components that have a good overlap with the ground truth. We will identify the connected components in the two masks and then match them based on the Dice coefficient.

In [ ]:
# Detail: cv.connectedComponents() requires an 8-bit image as input.
components = cv.connectedComponents(mask, connectivity=8)[1]
components_test = cv.connectedComponents(mask_test, connectivity=8)[1]

# Problem: We need to match the components in the two masks, as the component
#          IDs are arbitrary. We can do this by computing the Dice coefficient 
#          for each pair of components and then assign the component IDs based 
#          on the Dice coefficient.

# Compute the Dice coefficient for each pair of components.
dices = np.zeros((components.max()+1, components_test.max()+1))
mask_match = np.zeros_like(img, dtype=np.uint8)
for i in range(1, components.max()+1):
    for j in range(1, components_test.max()+1):
        mask_i = components == i
        mask_j = components_test == j
        dices[i, j] = dice(mask_i, mask_j)
        
# Method 1: Greedy matching: For each component of the ground truth, we assign
#           the component of the test mask with the highest Dice coefficient.
#           Note: It is possible 
mask_match = np.zeros_like(img, dtype=np.uint8)
for i in range(1, components.max()+1):
    j = np.argmax(dices[i, :])
    if dices[i, j] > 0.95:
        # Green: Very good match
        mask_match[components_test == j] = [0, 255, 0]
    elif dices[i, j] > 0.8:
        # Yellow: Good match
        mask_match[components_test == j] = [255, 255, 0]
    elif dices[i, j] > 0.0:
        # Red: Bad match
        mask_match[components_test == j] = [255, 0, 0]
        
# # Method 2: Solve the assignment problem using the Hungarian algorithm.
from scipy.optimize import linear_sum_assignment
row_ind, col_ind = linear_sum_assignment(-dices)
mask_match = np.zeros_like(img, dtype=np.uint8)
for i, j in zip(row_ind, col_ind):
    if dices[i, j] > 0.95:
        mask_match[components_test == j] = [0, 255, 0]
    elif dices[i, j] > 0.8:
        mask_match[components_test == j] = [255, 255, 0]
    elif dices[i, j] > 0.0:
        mask_match[components_test == j] = [255, 0, 0]
     

In [ ]:
print("Dice coefficients:")
print("="*25)
display(pd.DataFrame(dices).head(20).round(2))
isp.show_image(mask_match, suppress_info=True);

---

**TODO**: Mention Voronoi diagrams and Delaunay triangulation